In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import numpy as np
import random
from time import sleep, time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from typing import List, Dict, Optional, Tuple

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
def get_request(url, retries=5):
    list_of_user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15",
    ]
    headers = {
        "User-Agent": random.choice(list_of_user_agents),
        "Content-Type": "application/json",
        "Cache-Control": "no-cache",
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response
    elif retries > 0:
        print(f'Request failed with status code {response.status_code}. Retrying...')
        sleep(10)
        return get_request(url, retries-1)
    else:
        print(f'Request failed with status code {response.status_code} after {retries} retries.')
        return None

In [5]:
def scrape_lawyer_code(i):
    url = f'https://api.justika.com/lawyer/?page={i}'

    response = get_request(url)

    # using time sleep to prevent website blocking
    random_array = np.random.uniform(low=0.01, high=0.1, size=100)
    sleep(np.random.choice(random_array))

    json_response = response.json()
    lawyer_code_list = [result['code'] for result in json_response['results']]
    return lawyer_code_list

In [6]:
def get_final_lawyer_code():
    final_list = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        # Submit tasks to the executor
        futures = [executor.submit(scrape_lawyer_code, i) for i in range(1, 56)]

        # Collect the results as they complete
        for future in as_completed(futures):
            lawyer_code_list = future.result()
            final_list.extend(lawyer_code_list)

    return final_list

In [7]:
final_lawyer_code_list = get_final_lawyer_code()
# print(final_lawyer_code_list)

In [8]:
def scrape_lawyer_review(name, page):
    url = f'https://api.justika.com/lawyer/{name}/review/?page={page}'

    response = requests.get(url)

    # using time sleep to prevent website blocking
    random_array = np.random.uniform(low=0.01, high=0.1, size=100)
    sleep(np.random.choice(random_array))

    json_response = response.json()
    review_count_list = [json_response["count"]]
    return review_count_list

In [9]:
def get_final_lawyer_review(names):
    final_review_list = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        # Submit tasks to the executor
        futures = [executor.submit(scrape_lawyer_review, name, 1) for name in names]

        # Collect the results as they complete
        for future in as_completed(futures):
            try:
                review_count_list = future.result()
                final_review_list.extend(review_count_list)
            except Exception as e:
                print(f"An error occurred: {e}")

    return final_review_list

In [10]:
final_review_list = get_final_lawyer_review(final_lawyer_code_list)

In [11]:
def scrape_lawyer_review_data(name: str, page: int, page_size: int = 5) -> List[Dict]:
    """
    Scrape review data for a specific lawyer and page.

    Args:
        name: Lawyer identifier
        page: Page number to fetch
        page_size: Number of results per page

    Returns:
        List of dictionaries containing review data
    """
    url = f'https://api.justika.com/lawyer/{name}/review/?page={page}&page_size={page_size}'

    try:
        response = requests.get(url)
        response.raise_for_status()

        # Random delay to prevent rate limiting
        random_delay = np.random.uniform(0.01, 0.1)
        sleep(random_delay)

        json_response = response.json()

        if 'results' in json_response:
            return [{
                "order_code": result["order_code"],
                "post_date": result["post_date"],
                "order_category": result["order_category"],
                "order_type": result["order_type"],
                "order_type_name": result["order_type_name"],
                "score": result["score"],
                "feedback": result["feedback"],
                "lawyer_code": name
            } for result in json_response['results']]

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for lawyer {name}, page {page}: {e}")
    except Exception as e:
        print(f"Unexpected error for lawyer {name}, page {page}: {e}")

    return []

In [12]:
def get_lawyer_total_pages(name: str, page_size: int = 5) -> int:
    """
    Get the total number of pages for a lawyer based on their first page response.

    Args:
        name: Lawyer identifier/name
        page_size: Number of results per page

    Returns:
        Total number of pages
    """
    url = f'https://api.justika.com/lawyer/{name}/review/?page=1&page_size={page_size}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        total_reviews = response.json().get('count', 0)
        return (total_reviews + page_size - 1) // page_size
    except Exception as e:
        print(f"Error getting page count for lawyer {name}: {e}")
        return 0

In [13]:
def process_lawyer(name: str, page_size: int = 5) -> List[Dict]:
    """
    Process all pages for a single lawyer.

    Args:
        name: Lawyer identifier/name
        page_size: Number of results per page

    Returns:
        List of review data for all pages
    """
    max_pages = get_lawyer_total_pages(name, page_size)
    all_reviews = []

    if max_pages > 0:
        print(f"Processing {max_pages} pages for lawyer {name}")
        for page in range(1, max_pages + 1):
            page_data = scrape_lawyer_review_data(name, page, page_size)
            all_reviews.extend(page_data)
            print(f"Completed page {page}/{max_pages} for lawyer {name}")

    return all_reviews

In [14]:
def get_final_lawyer_review_data(lawyer_names: List[str], page_size: int = 5) -> List[Dict]:
    """
    Fetch all review data for multiple lawyers using parallel processing.

    Args:
        lawyer_names: List of lawyer names/identifiers
        page_size: Number of results per page

    Returns:
        List of all review data dictionaries
    """
    final_review_data_list = []

    # Process lawyers in parallel
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Submit all tasks
        future_to_lawyer = {
            executor.submit(process_lawyer, name, page_size): name
            for name in lawyer_names
        }

        # Process completed tasks
        for future in as_completed(future_to_lawyer):
            lawyer_name = future_to_lawyer[future]
            try:
                lawyer_data = future.result()
                final_review_data_list.extend(lawyer_data)
                print(f"Completed processing all data for lawyer {lawyer_name}")
            except Exception as e:
                print(f"Failed to process lawyer {lawyer_name}: {e}")

    return final_review_data_list

In [15]:
final_review_data_list = get_final_lawyer_review_data(final_lawyer_code_list)

Processing 26 pages for lawyer nasrul-saftiar
Completed page 1/26 for lawyer nasrul-saftiar
Processing 60 pages for lawyer muhammad-hamzah
Processing 20 pages for lawyer hanna-marissa-sh-mcommerce
Processing 10 pages for lawyer ferdy-rizky-adilya-sh-mh-cla
Processing 6 pages for lawyer ari-irfano
Processing 21 pages for lawyer putu-bravo-timothy-sh-mh
Processing 13 pages for lawyer lucky-omega-hasan
Completed page 1/60 for lawyer muhammad-hamzah
Processing 27 pages for lawyer rizky-rahmawati-pasaribu-sh-llm
Processing 286 pages for lawyer mohammad-agus-riza-hufaida-sh
Processing 125 pages for lawyer andrian-febrianto
Completed page 1/10 for lawyer ferdy-rizky-adilya-sh-mh-cla
Completed page 1/20 for lawyer hanna-marissa-sh-mcommerce
Completed page 1/6 for lawyer ari-irfano
Completed page 1/27 for lawyer rizky-rahmawati-pasaribu-sh-llm
Completed page 1/286 for lawyer mohammad-agus-riza-hufaida-sh
Completed page 1/21 for lawyer putu-bravo-timothy-sh-mh
Completed page 1/13 for lawyer luck

In [16]:
# Convert to DataFrame
df2 = pd.DataFrame(final_review_data_list)
df2

,order_code,post_date,order_category,order_type,order_type_name,score,feedback,lawyer_code
0,5616a74,2024-03-12,None,call,Konsultasi via Telepon,5,None,ari-irfano
1,e7d56d7,2023-12-21,Lainnya,CHAT,Konsultasi via Chat,5,Jawaban sangat membantu dan bermanfaat. Terima...,ari-irfano
2,e11eafd,2023-12-21,Pidana dan Laporan Polisi,CHAT,Konsultasi via Chat,5,sangat membantu jawabannya,ari-irfano
3,654998e,2023-12-20,Pertanahan dan Properti,CHAT,Konsultasi via Chat,5,"Terima kasih untuk penjelasan nya pak ari, say...",ari-irfano
4,3b778de,2023-12-19,Lainnya,CHAT,Konsultasi via Chat,5,,ari-irfano
...,...,...,...,...,...,...,...,...
12674,84adf7f,2020-08-12,None,CHAT,Konsultasi via Chat,5,,alfred-nobel-sugio-hartono
12675,1d531d4,2020-08-11,None,CHAT,Konsultasi via Chat,5,terima kasih,alfred-nobel-sugio-hartono
12676,1682116,2020-07-13,None,call,Konsultasi via Telepon,5,None,alfred-nobel-sugio-hartono
12677,3be76e3,2020-07-03,Keluarga,call,Konsultasi via Telepon,5,None,alfred-nobel-sugio-hartono


In [17]:
df2.to_excel('/content/gdrive/MyDrive/Colab Notebooks/Justika/web-scraping.xlsx')